In [4]:
import polars as pl
import joblib
from src.data_loader import get_all_data, split_date_lazy
from src.feature_engineering import build_features_from_purchases, build_labels
from src.trainer import train_model
from src import config


In [5]:
# ======================================================================
# 2. LOAD & SPLIT DATA USING CUSTOM UTILITY
# ======================================================================
print("📂 Loading and splitting data...")

# 1. Load dữ liệu
lf_items, lf_users, lf_purchases = get_all_data("/home/genyonguyen/Data/CS116_Reccommender_System/data")

# 2. Sử dụng hàm của bạn để chia tập
# Hàm này tự động xử lý cast datetime nếu created_date đang là String
lf_purchase_hist, lf_purchase_recent, lf_purchase_val = split_date_lazy(
    lf_purchases, 
    date_column_name="created_date"
)

# 3. Kiểm tra nhanh kết quả bằng collect_schema (không tốn RAM)
print("✅ Data split completed.")
print(f"   - Hist samples: {lf_purchase_hist.select(pl.len()).collect().item():,}")
print(f"   - Recent samples: {lf_purchase_recent.select(pl.len()).collect().item():,}")
print(f"   - Val samples: {lf_purchase_val.select(pl.len()).collect().item():,}")

📂 Loading and splitting data...
📂 Đang quét dữ liệu từ thư mục: /home/genyonguyen/Data/CS116_Reccommender_System/data
✅ Đã load Items
✅ Đã load Users
✅ Đã load Purchases
✅ Data split completed.
   - Hist samples: 29,580,965
   - Recent samples: 3,099,667
   - Val samples: 3,049,193


In [3]:
# Kiểm tra xem có bao nhiêu user xuất hiện ở cả Hist và Recent
user_hist = lf_purchase_hist.select("customer_id").unique().collect()
user_recent = lf_purchase_recent.select("customer_id").unique().collect()

intersection = user_hist.join(user_recent, on="customer_id", how="inner")
print(f"Số user trong Hist: {user_hist.height:,}")
print(f"Số user trong Recent: {user_recent.height:,}")
print(f"Số user giao thoa: {intersection.height:,}")

Số user trong Hist: 2,170,516
Số user trong Recent: 650,383
Số user giao thoa: 507,012


In [6]:
# ======================================================================
# 3. INITIALIZE CORE DATA & CO-OCCURRENCE (FIXED FOR SCHEMA)
# ======================================================================
import os
import joblib
import polars as pl

print("🏗️ Đang chuẩn bị dữ liệu nền tảng cho Model và Inference...")

# 1. Gộp toàn bộ lịch sử mua hàng 
# LƯU Ý: Phải gộp cả lf_purchase_hist nếu muốn tính X1-X3 đầy đủ nhất
full_history = pl.concat([lf_purchase_val])

# 2. Nạp Ground Truth
gt_path = "groundtruth_main.pkl"
if os.path.exists(gt_path):
    gt_raw = joblib.load(gt_path)
    target_users_set = set(str(u) for u in gt_raw.keys())
    print(f" ✅ Đã nạp Ground Truth: {len(target_users_set):,} users.")
else:
    print(" ⚠️ Cảnh báo: Không tìm thấy file groundtruth_main.pkl!")

# 3. Tạo bảng lịch sử mua hàng rút gọn
# SỬA LỖI: item_id là String, customer_id là Int32 theo Schema của bạn
user_items = (
    full_history
    .select([
        pl.col("customer_id").cast(pl.Int32), # Khớp Schema Int32
        pl.col("item_id").cast(pl.String)     # Khớp Schema String
    ])
    .drop_nulls()
    .unique() 
    .group_by("customer_id")
    .agg(pl.col("item_id").alias("item_id_list"))
    .collect()
)

# 4. Tính toán Ma trận đồng xuất hiện (top_item_links)
print(" 🛠️ Đang tính toán Ma trận Co-occurrence...")
user_items_clean = user_items.explode("item_id_list").rename({"item_id_list": "item_id"})

# Self-join để tìm các cặp sản phẩm mua cùng nhau
item_pairs = (
    user_items_clean.join(user_items_clean, on="customer_id", suffix="_right")
    .filter(pl.col("item_id") < pl.col("item_id_right")) # Tránh trùng lặp cặp (A,B) và (B,A)
    .group_by(["item_id", "item_id_right"])
    .len()
    .filter(pl.col("len") > 2) 
)

top_item_links = (
    pl.concat([
        item_pairs.select([pl.col("item_id"), pl.col("item_id_right"), "len"]),
        item_pairs.select([pl.col("item_id_right").alias("item_id"), pl.col("item_id").alias("item_id_right"), "len"])
    ])
    .sort(["item_id", "len"], descending=[False, True])
    .group_by("item_id")
    .head(12) 
    .select([
        pl.col("item_id").alias("item_id_source"), 
        pl.col("item_id_right").alias("item_id_rec")
    ])
)

# 5. Tạo đặc trưng phục vụ Inference (infer_features_df)
print(" 🛠️ Đang xây dựng infer_features_df từ full_history...")
# Hàm này sẽ tự động nhận diện item_id là String và customer_id là Int32
infer_features_df = build_features_from_purchases(
    lf_purchases=full_history, 
    lf_items=lf_items, 
    lf_users=lf_users
)

print("\n🚀 KHỞI TẠO XONG! Các biến đã khớp Schema String/Int32.")

🏗️ Đang chuẩn bị dữ liệu nền tảng cho Model và Inference...
 ⚠️ Cảnh báo: Không tìm thấy file groundtruth_main.pkl!
 🛠️ Đang tính toán Ma trận Co-occurrence...
 🛠️ Đang xây dựng infer_features_df từ full_history...

🚀 KHỞI TẠO XONG! Các biến đã khớp Schema String/Int32.


In [5]:
# ======================================================================
# 4. BUILDING TRAIN DATASET (CHUẨN HÓA THEO SCHEMA)
# ======================================================================

def build_features_from_purchases_fixed(lf_purchases, lf_items, lf_users=None):
    # 1. JOIN metadata - Chú ý: item_id đều là String nên Join rất an toàn
    data_lf = lf_purchases.join(
        lf_items.select(['item_id', 'brand', 'age_group', 'category']),
        on='item_id',
        how='inner'
    )

    # 2. TÍNH TOÁN VÀ GIỮ LẠI CỘT KEY
    feature_df = (
        data_lf
        .select([
            pl.col("customer_id").alias("X_-1"), # Giữ Int32 theo Schema
            pl.col("item_id").alias("X_0"),      # Giữ String theo Schema
            pl.col("brand"),                     # String
            pl.col("category"),                  # String
            pl.col("age_group"),                 # String

            # Tính toán tần suất
            pl.len().over(["customer_id", "brand"]).alias("X_1"),
            pl.len().over(["customer_id", "age_group"]).alias("X_2"),
            pl.len().over(["customer_id", "category"]).alias("X_3"),
        ])
        .unique(subset=["X_-1", "X_0"])
        .with_columns([
            pl.col("X_1").cast(pl.Float64),
            pl.col("X_2").cast(pl.Float64),
            pl.col("X_3").cast(pl.Float64),
        ])
    )
    return feature_df

print("🛠 Building features from Hist...")
train_features_df = build_features_from_purchases_fixed(lf_purchase_hist, lf_items)

print("🎯 Building labels from Recent...")
train_label_df = build_labels(lf_purchase_hist, lf_purchase_recent, lf_items, negative_ratio=1.0)

print("🔗 Merging via Entity Profile...")
# Chú ý: Lấy Metadata gốc (String) để Join
train_label_with_meta = train_label_df.join(
    lf_items.select(["item_id", "brand", "category", "age_group"]),
    on="item_id", 
    how="left"
)

# Bước 4: Tách profile (Không cast Int64 cho brand/category nữa)
user_brand_feat = train_features_df.select(["X_-1", "brand", "X_1"]).unique(subset=["X_-1", "brand"])
user_age_feat   = train_features_df.select(["X_-1", "age_group", "X_2"]).unique(subset=["X_-1", "age_group"])
user_cat_feat   = train_features_df.select(["X_-1", "category", "X_3"]).unique(subset=["X_-1", "category"])

# Bước 5: Join tổng hợp
train_df = (
    train_label_with_meta
    .select([
        pl.col("customer_id").alias("X_-1"),
        pl.col("item_id").alias("X_0"),
        pl.col("label").alias("Y"),
        "brand", "category", "age_group"
    ])
    .join(user_brand_feat, on=["X_-1", "brand"], how="left")
    .join(user_age_feat, on=["X_-1", "age_group"], how="left")
    .join(user_cat_feat, on=["X_-1", "category"], how="left")
    .with_columns(
        pl.col(["X_1", "X_2", "X_3"]).fill_null(0)
    )
    .select(["X_-1", "X_0", "X_1", "X_2", "X_3", "Y"])
)

print(f"✅ Train dataset created: {train_df.select(pl.len()).collect().item():,} rows.")
display(train_df.limit(5).collect())

🛠 Building features from Hist...
🎯 Building labels from Recent...
🚀 Building Labels with Vectorized Hard Negative Strategy...
🔗 Merging via Entity Profile...
✅ Train dataset created: 18,765,984 rows.


X_-1,X_0,X_1,X_2,X_3,Y
i32,str,f64,f64,f64,i8
5624205,"""0007160000078""",0.0,99.0,0.0,1
6642730,"""5481000000002""",0.0,63.0,0.0,1
5777942,"""6849000000003""",3.0,21.0,3.0,1
6288770,"""5952000000001""",0.0,41.0,0.0,1
5415343,"""0952000000074""",1.0,8.0,0.0,1


In [6]:
# ======================================================================
# CHECK DATA DENSITY (KIỂM TRA SỐ DÒNG BẰNG 0)
# ======================================================================

# Tính toán thống kê
check_df = train_df.select([
    pl.len().alias("total_rows"),
    # Đếm số dòng mà cả 3 đặc trưng đều bằng 0
    pl.struct(["X_1", "X_2", "X_3"])
      .filter((pl.col("X_1") == 0) & (pl.col("X_2") == 0) & (pl.col("X_3") == 0))
      .count().alias("all_zeros"),
    
    # Đếm riêng lẻ từng cột
    (pl.col("X_1") == 0).sum().alias("X_1_zero"),
    (pl.col("X_2") == 0).sum().alias("X_2_zero"),
    (pl.col("X_3") == 0).sum().alias("X_3_zero")
]).collect()

# Trích xuất giá trị
total = check_df["total_rows"][0]
all_zero = check_df["all_zeros"][0]
x1_z = check_df["X_1_zero"][0]
x2_z = check_df["X_2_zero"][0]
x3_z = check_df["X_3_zero"][0]

print(f"📊 THỐNG KÊ TRAIN_DF (Tổng số dòng: {total:,})")
print("-" * 45)
print(f"🚫 Số dòng có X1, X2, X3 đều bằng 0: {all_zero:,} ({all_zero/total*100:.2f}%)")
print(f"🔸 Cột X_1 (Brand) bằng 0: {x1_z:,} ({x1_z/total*100:.2f}%)")
print(f"🔸 Cột X_2 (Age Group) bằng 0: {x2_z:,} ({x2_z/total*100:.2f}%)")
print(f"🔸 Cột X_3 (Category) bằng 0: {x3_z:,} ({x3_z/total*100:.2f}%)") 
print("-" * 45)

if all_zero / total > 0.8:
    print("⚠️ CẢNH BÁO: Tỷ lệ dòng trống quá cao! Hãy kiểm tra lại logic tạo Candidate.")
else:
    print("✅ Tỷ lệ dữ liệu ổn định. Bạn có thể tiến hành huấn luyện.")

📊 THỐNG KÊ TRAIN_DF (Tổng số dòng: 18,766,176)
---------------------------------------------
🚫 Số dòng có X1, X2, X3 đều bằng 0: 695,393 (3.71%)
🔸 Cột X_1 (Brand) bằng 0: 4,761,292 (25.37%)
🔸 Cột X_2 (Age Group) bằng 0: 4,276,069 (22.79%)
🔸 Cột X_3 (Category) bằng 0: 1,698,387 (9.05%)
---------------------------------------------
✅ Tỷ lệ dữ liệu ổn định. Bạn có thể tiến hành huấn luyện.


In [3]:
# ======================================================================
# 5. TRAINING MODEL (OPTIMIZED FOR X1, X2, X3 & SCHEMA)
# ======================================================================
import xgboost as xgb
import joblib
import os
import polars as pl
from src import config

In [3]:
model_xgb = joblib.load(config.MODEL_PATH)

In [ ]:
# Danh sách feature bạn đã chọn (X1: Brand, X2: Age, X3: Cat)
feature_cols = ["X_1", "X_2", "X_3"] 

# Thực thi huấn luyện
model_xgb = train_model(train_df, feature_cols)

# 4. Lưu mô hình
if not os.path.exists('model'):
    os.makedirs('model')

joblib.dump(model_xgb, config.MODEL_PATH)
print(f"✅ Model trained and saved at: {config.MODEL_PATH}")

⏳ Collecting and processing training data...


KeyboardInterrupt: 

In [8]:
import matplotlib.pyplot as plt

# Lấy điểm quan trọng của các tính năng
importance = model_xgb.get_score(importance_type='gain') # Hoặc 'weight'
sorted_importance = sorted(importance.items(), key=lambda x: x[1], reverse=True)

# Hiển thị
print("📊 TẦM QUAN TRỌNG CỦA CÁC TÍNH NĂNG (FEATURE IMPORTANCE):")
for feat, score in sorted_importance:
    print(f"  - {feat}: {score:.2f}")

📊 TẦM QUAN TRỌNG CỦA CÁC TÍNH NĂNG (FEATURE IMPORTANCE):
  - X_3: 4404.66
  - X_1: 873.54
  - X_2: 678.32


In [4]:
import pickle as pkl

gt = pkl.load(open("groundtruth.pkl", "rb"))
target_users_set = set(gt.keys())

In [5]:
warm_users_set = set(
    lf_purchase_val
    .select("customer_id")
    .unique()
    .collect(engine="streaming")
    .to_series()
    .to_list()
) & target_users_set

In [6]:
from collections import defaultdict

df_purchase_val = (
    lf_purchase_val
    .select(["customer_id", "item_id"])
    .join(lf_items.select(["item_id", "brand", "age_group", "category"]), on="item_id", how="left")
    .collect(engine="streaming")
)

# Aggregate counts for each attribute in parallel
brand_counts = (
    df_purchase_val.group_by(['customer_id', 'brand'])
    .agg(pl.len().alias('count'))
)

age_counts = (
    df_purchase_val.group_by(['customer_id', 'age_group'])
    .agg(pl.len().alias('count'))
)

category_counts = (
    df_purchase_val.group_by(['customer_id', 'category'])
    .agg(pl.len().alias('count'))
)

# Build the nested defaultdict structure
feature_dict = defaultdict(lambda: {
    'brand': {},
    'age_group': {},
    'category': {}
})

# Populate brand counts
for row in brand_counts.iter_rows(named=True):
    feature_dict[row['customer_id']]['brand'][row['brand']] = row['count']

# Populate age_group counts
for row in age_counts.iter_rows(named=True):
    feature_dict[row['customer_id']]['age_group'][row['age_group']] = row['count']

# Populate category counts
for row in category_counts.iter_rows(named=True):
    feature_dict[row['customer_id']]['category'][row['category']] = row['count']

del df_purchase_val
del brand_counts
del age_counts
del category_counts

In [21]:
item_feature_dict = (
    lf_items
    .with_columns(
        pl.concat_str(["brand", "age_group", "category"], separator="|")
        .str.split("|")
        .alias("values")
    )
    .select(["item_id", "values"])
    .collect()
    .to_dict(as_series=False)
)

item_feature_dict = dict(zip(item_feature_dict["item_id"], item_feature_dict["values"]))

In [27]:
# ======================================================================
# 6. RUN INFERENCE FOR WARM USERS (OPTIMIZED WITH ABSOLUTE BOOSTING)
# ======================================================================
import xgboost as xgb
import polars as pl
import numpy as np
from src.candidate_selector import CandidateSelector
from tqdm import tqdm

customer_ids = []
pred_ids = []

purchases_recent_val_lf = pl.concat([lf_purchase_recent, lf_purchase_val])
cs = CandidateSelector(purchases_recent_val_lf, lf_users, lf_items)

# --- BƯỚC 3: DỰ ĐOÁN VÀ RERANK THEO BATCH ---
for customer_id in tqdm(warm_users_set):

    candidates = cs.get_candidates(customer_id)

    # Feature extract
    features = []
    for c_id in candidates:
        brand_count = feature_dict[customer_id]["brand"].get(item_feature_dict[c_id][0], 0)
        age_count = feature_dict[customer_id]["age_group"].get(item_feature_dict[c_id][1], 0)
        category_count = feature_dict[customer_id]["category"].get(item_feature_dict[c_id][2], 0)
        features.append([brand_count, age_count, category_count])

    X_infer = pl.DataFrame(features, schema=["X_1", "X_2", "X_3"], orient='row')
    probs = model_xgb.predict(xgb.DMatrix(X_infer))

    ranked_indices = np.argsort(probs)[::-1]
    ranked_results = []
    
    # Rerank
    for idx in ranked_indices[:10]:
        ranked_results.append(candidates[idx])
    
    customer_ids.extend([customer_id] * len(ranked_results))
    pred_ids.extend(ranked_results)


recommendations_warm = pl.DataFrame({"X_-1": customer_ids, "X_0": pred_ids})

[Build lookup structures] Completed in 2.89 seconds
[Build item co-purchase matrix] Completed in 2.55 seconds
[Compute item popularity] Completed in 0.59 seconds
[Build item similarity map] Completed in 7.54 seconds
[Collect item features] Completed in 0.07 seconds
[Collect user purchases with features] Completed in 0.57 seconds
[Cache business rule candidates] Completed in 0.58 seconds
[Build user reorder candidates] Completed in 0.79 seconds


664277it [00:00, 1105148.84it/s]


[ERROR] Timer was not started.


100%|██████████| 244990/244990 [2:00:48<00:00, 33.80it/s]  


In [25]:
import json

recommendations_warm = json.load(open("recommendations_warm.json", "rb"))

In [26]:
recommendations_warm = {int(k): list(map(str, v)) for k, v in recommendations_warm.items()}

In [6]:
hist_dict = (
    lf_purchase_val.select(["customer_id", "item_id"])
    .group_by("customer_id")
    .agg(pl.col("item_id"))
    .collect()
    .to_dict(as_series=False)
)

In [7]:
hist_dict = dict(zip(hist_dict["customer_id"], hist_dict["item_id"]))

In [8]:
import pickle as pkl

gt = pkl.load(open("groundtruth.pkl", "rb"))

In [ ]:
import numpy as np
from tqdm import tqdm

def precision_at_k_fast(pred, gt, hist, filter_bought_items=True, K=10):
    precisions = []
    cold_start_users = []

    # users that have both prediction and history
    valid_users = gt.keys() & pred.keys() & hist.keys()

    skipped = 0
    for user in tqdm(gt.keys()):
        if user not in valid_users:
            cold_start_users.append(user)
            continue

        # ground truth as set
        relevant_items = set(gt[user])

        if filter_bought_items:
            relevant_items.difference_update(hist[user])

        if not relevant_items:
            skipped += 1
            precisions.append(0.0)
            continue

        # prediction@K
        pred_topk = pred[user][:K]
        hits = 0
        for item in pred_topk:
            if item in relevant_items:
                hits += 1

        precisions.append(hits / K)
    
    print(f"Skipped: {skipped}")

    return np.mean(precisions), cold_start_users

In [30]:
score, _ = precision_at_k_fast(recommendations_warm, gt, hist_dict)
score

100%|██████████| 391900/391900 [00:00<00:00, 711886.98it/s]

Skipped: 47470


np.float64(0.004439773051961304)

In [32]:
score, _ = precision_at_k_fast(recommendations_warm, gt, hist_dict, filter_bought_items=False)
score

100%|██████████| 391900/391900 [00:00<00:00, 1053644.60it/s]

Skipped: 0


np.float64(0.004439773051961304)

In [28]:
# ======================================================================
# 5. CREATE POPULAR ITEMS LIST (FOR COLD START STRATEGY)
# ======================================================================

print("📦 Đang chuẩn bị danh sách Popular Items từ dữ liệu gần nhất...")

# Nên sử dụng dữ liệu của tháng gần nhất (lf_purchase_val) để lấy xu hướng mới nhất
# Điều này giúp tăng Precision cho nhóm Cold lên mức ~0.011668
popular_items_list = (
    lf_purchase_val.group_by("item_id")
    .agg(pl.len().alias("count"))
    .sort("count", descending=True)
    .head(100) # Lấy 100 món để làm kho dự phòng (Candidate Pool)
    .collect()
    .get_column("item_id")
    .to_list()
)

# Ép kiểu sang String để đồng nhất với schema của Submission và Item-Links
popular_items_list_str = [str(i) for i in popular_items_list]

print(f"✅ Đã tạo xong danh sách {len(popular_items_list_str)} món phổ biến nhất.")
print(f"📍 Ví dụ 5 món đầu bảng: {popular_items_list_str[:5]}")

📦 Đang chuẩn bị danh sách Popular Items từ dữ liệu gần nhất...
✅ Đã tạo xong danh sách 100 món phổ biến nhất.
📍 Ví dụ 5 món đầu bảng: ['4690000000001', '1512000000004', '2803000000013', '6768000000005', '0020020000185']


In [38]:
# ======================================================================
# 7. HYBRID HANDLING (REPURCHASE + ITEM-LINKS + POPULAR) - FIXED VERSION
# ======================================================================
import os
import polars as pl

print("[1/3] Phân tách nhóm Warm và Cold Users...")

# 1. Thu thập kết quả từ Cell 6
if isinstance(recommendations_warm, pl.LazyFrame):
    warm_results = recommendations_warm.select([
        pl.col("X_-1").cast(pl.String),
        pl.col("X_0").cast(pl.String)
    ]).collect()
else:
    warm_results = recommendations_warm.select([
        pl.col("X_-1").cast(pl.String),
        pl.col("X_0").cast(pl.String)
    ])

cold_users_ids = list(target_users_set - warm_users_set)
print(f" ✅ Thống kê: {len(warm_users_set & target_users_set):,} Warm users và {len(cold_users_ids):,} Cold users.")

# --- 2. Sinh gợi ý bổ trợ đa tầng ---
print("[2/3] Đang tạo gợi ý bổ trợ (Repurchase & Item-Links)...")
df_cold_base = pl.DataFrame({"X_-1": [str(u) for u in cold_users_ids]}).lazy()

# Nhánh A: Repurchase (Đồ cũ cho 47k users)
batch_repur = (
    df_cold_base.join(
        user_items.lazy().with_columns(pl.col("customer_id").cast(pl.String)).rename({"customer_id": "X_-1"}),
        on="X_-1", how="inner"
    )
    .explode("item_id_list")
    .select([pl.col("X_-1"), pl.col("item_id_list").alias("X_0")])
)

# Nhánh B: Semi-cold (Đồ mua kèm)
batch_semi = (
    df_cold_base.join(
        user_items.lazy().with_columns(pl.col("customer_id").cast(pl.String)).rename({"customer_id": "X_-1"}), 
        on="X_-1", how="inner"
    )
    .explode("item_id_list")
    .rename({"item_id_list": "item_id"})
    .join(
        top_item_links.lazy().with_columns([
            pl.col("item_id_source").cast(pl.String),
            pl.col("item_id_rec").cast(pl.String)
        ]), 
        left_on="item_id", right_on="item_id_source", how="inner"
    )
    .select([pl.col("X_-1"), pl.col("item_id_rec").alias("X_0")])
)

# Nhánh C: Pure-cold (Popular)
users_with_any_rec = pl.concat([
    batch_semi.select("X_-1").unique(),
    batch_repur.select("X_-1").unique()
]).unique()

batch_pure = (
    df_cold_base.join(users_with_any_rec, on="X_-1", how="anti")
    .with_columns(pl.lit(popular_items_list_str).alias("X_0"))
    .explode("X_0")
    .select(["X_-1", "X_0"])
)

# Gộp kết quả Cold Start với thứ tự ưu tiên
recommendations_cold = pl.concat([
    batch_repur.with_columns(pl.lit(1).alias("sub_prio")),
    batch_semi.with_columns(pl.lit(2).alias("sub_prio")),
    batch_pure.with_columns(pl.lit(3).alias("sub_prio"))
]).unique(subset=["X_-1", "X_0"]).sort(["X_-1", "sub_prio"]).collect()

# --- 3. KẾT HỢP VÀ XUẤT SUBMISSION (ÉP THEO GT_DICT) ---
print("[3/3] Đang gộp kết quả và lọc theo đúng danh sách mục tiêu...")

# Lấy danh sách ID chuẩn từ Ground Truth (Đáp án của thầy)
final_target_ids = list(map(str, list(gt.keys())))

submission = (
    pl.concat([
        warm_results.with_columns(pl.lit(1).alias("prio")),
        recommendations_cold.select(["X_-1", "X_0"]).with_columns(pl.lit(2).alias("prio"))
    ])
    .cast({"X_-1": pl.String})
    # BƯỚC QUAN TRỌNG: Chỉ giữ lại những người có trong đáp án thực tế
    .filter(pl.col("X_-1").is_in(final_target_ids)) 
    .unique(subset=["X_-1", "X_0"]) 
    .sort(["X_-1", "prio"]) 
    .group_by("X_-1")
    .agg(pl.col("X_0").head(12)) 
    .with_columns(
        pl.col("X_0").list.join(" ").alias("prediction")
    )
    .select([
        pl.col("X_-1").alias("customer_id"),
        "prediction"
    ])
    .unique(subset=["customer_id"]) 
)

# Ghi file
submission.write_csv("final_submission.csv")

print(f" ✅ TẤT CẢ HOÀN TẤT!")
print(f" 📊 Tổng số User thực tế trong file: {submission.height:,}")

[1/3] Phân tách nhóm Warm và Cold Users...
 ✅ Thống kê: 244,990 Warm users và 146,910 Cold users.
[2/3] Đang tạo gợi ý bổ trợ (Repurchase & Item-Links)...
[3/3] Đang gộp kết quả và lọc theo đúng danh sách mục tiêu...
 ✅ TẤT CẢ HOÀN TẤT!
 📊 Tổng số User thực tế trong file: 391,900


In [40]:
import pickle as pkl

# Nạp file pkl chứa đáp án thực tế
print("📂 Đang nạp Ground Truth từ file groundtruth_main.pkl...")
gt_dict_raw = pkl.load(open("groundtruth.pkl", "rb"))

# Chuyển đổi key sang String để đảm bảo khớp với customer_id trong submission
gt_dict = {str(k): v for k, v in gt_dict_raw.items()}

print(f"✅ Đã nạp xong {len(gt_dict):,} users trong Ground Truth.")

📂 Đang nạp Ground Truth từ file groundtruth_main.pkl...
✅ Đã nạp xong 391,900 users trong Ground Truth.


In [41]:
import polars as pl

# 1. Đọc lại file kết quả vừa xuất ra
df_pred = pl.read_csv("final_submission.csv")

# 2. Chuyển đổi từ bảng sang Dictionary để tra cứu nhanh trong hàm đánh giá
# Format: { "customer_id": ["item1", "item2", ... , "item12"] }
pred_dict = {
    str(row["customer_id"]): row["prediction"].split(" ") 
    for row in df_pred.iter_rows(named=True)
}

print(f"✅ Đã chuẩn bị pred_dict cho {len(pred_dict):,} users.")

✅ Đã chuẩn bị pred_dict cho 391,900 users.


In [42]:
# --- TẠO HIST_LOOKUP (SỬA LỖI .COLLECT) ---
print("🔍 Đang tạo hist_lookup từ dữ liệu lịch sử...")

# Kiểm tra nếu user_items là LazyFrame thì mới dùng .collect()
if isinstance(user_items, pl.LazyFrame):
    user_items_df = user_items.select(["customer_id", "item_id_list"]).collect()
else:
    user_items_df = user_items.select(["customer_id", "item_id_list"])

# Tạo Dictionary để tra cứu
hist_lookup = {
    str(row["customer_id"]): set(map(str, row["item_id_list"]))
    for row in user_items_df.iter_rows(named=True)
}

print(f"✅ Đã chuẩn bị xong lịch sử cho {len(hist_lookup):,} khách hàng.")

🔍 Đang tạo hist_lookup từ dữ liệu lịch sử...
✅ Đã chuẩn bị xong lịch sử cho 628,069 khách hàng.


In [43]:
import polars as pl
import numpy as np

def evaluate_flexible(pred_dict, gt_dict, hist_lookup, warm_users_trained, K=10, include_repurchase=True):
    warm_precs = []
    semi_precs = []
    pure_precs = []
    
    # 47k users này thường thuộc nhóm Warm/Semi-cold vì họ có lịch sử
    repurchase_only_count = 0 

    for user_id, gt_items in gt_dict.items():
        u_str = str(user_id)
        current_gt = set(map(str, gt_items))
        past_items = hist_lookup.get(u_str, set())
        
        # Xác định tập đồ mới
        relevant_new_items = current_gt - past_items
        
        # LOGIC SIÊU THAM SỐ:
        if not relevant_new_items:
            if not include_repurchase:
                # Tắt: Bỏ qua 47k users chỉ mua đồ cũ
                continue 
            else:
                # Bật: Coi đồ cũ khách mua lại là mục tiêu đánh giá (Ground Truth)
                target_items = current_gt
                repurchase_only_count += 1
        else:
            # Luôn tính những người có mua đồ mới
            target_items = relevant_new_items

        # Tính Precision@K
        user_preds = pred_dict.get(u_str, [])[:K]
        if not user_preds:
            precision = 0.0
        else:
            hits = len(set(user_preds) & target_items)
            precision = hits / K
        
        # Phân luồng báo cáo dựa trên trạng thái User
        if u_str in warm_users_trained:
            warm_precs.append(precision)
        elif u_str in hist_lookup:
            semi_precs.append(precision)
        else:
            pure_precs.append(precision)
            
    return warm_precs, semi_precs, pure_precs, repurchase_only_count

# --- CHẠY THỬ NGHIỆM ---
# Bạn có thể thay đổi include_repurchase = True hoặc False ở đây
include_47k = True 

warm_p, semi_p, pure_p, re_count = evaluate_flexible(
    pred_dict=pred_dict, 
    gt_dict=gt_dict, 
    hist_lookup=hist_lookup,
    warm_users_trained=warm_users_set,
    K=10,
    include_repurchase=include_47k
)

# --- IN BÁO CÁO ---
status = "BẬT (Tính cả 47k user mua đồ cũ)" if include_47k else "TẮT (Chỉ tính đồ mới)"
print(f"\n🚀 CHẾ ĐỘ ĐÁNH GIÁ: {status}")
print("="*60)
print(f"🔥 NHÓM WARM      : {len(warm_p):,} users - P@10: {np.mean(warm_p):.6f}")
print(f"⚡ NHÓM SEMI-COLD : {len(semi_p):,} users - P@10: {np.mean(semi_p):.6f}")
print(f"❄️ NHÓM PURE-COLD : {len(pure_p):,} users - P@10: {np.mean(pure_p):.6f}")
print("-" * 60)
total_evaluated = len(warm_p) + len(semi_p) + len(pure_p)
print(f"🏆 GLOBAL MEAN P@10 : {np.mean(warm_p + semi_p + pure_p):.6f}")
print(f"📦 Tổng User được tính: {total_evaluated:,} / {len(gt_dict):,}")
if include_47k:
    print(f"ℹ️ Trong đó có {re_count:,} users là nhóm 'Chỉ mua lại đồ cũ'.")
print("="*60)


🚀 CHẾ ĐỘ ĐÁNH GIÁ: BẬT (Tính cả 47k user mua đồ cũ)
🔥 NHÓM WARM      : 0 users - P@10: nan
⚡ NHÓM SEMI-COLD : 244,990 users - P@10: 0.005122
❄️ NHÓM PURE-COLD : 146,910 users - P@10: 0.005982
------------------------------------------------------------
🏆 GLOBAL MEAN P@10 : 0.005444
📦 Tổng User được tính: 391,900 / 391,900
ℹ️ Trong đó có 47,470 users là nhóm 'Chỉ mua lại đồ cũ'.


/home/genyonguyen/.local/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/genyonguyen/.local/lib/python3.13/site-packages/numpy/_core/_methods.py:144: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
